In [3]:
!pip install -qU pip seaborn pandas pyathena numexpr bottleneck

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.4 which is incompatible.
panel 0.13.1 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.3.2 which is incompatible.
sagemaker 2.199.0 requires urllib3<1.27, but you have urllib3 2.0.7 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.


In [4]:
import boto3
from pyathena import connect
import csv

import pandas as pd
import io

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [5]:
s3 = boto3.client('s3')

In [6]:
bucket_name = 'aai-540-final-data'

In [7]:
local_folder = 'data/'

In [8]:
# Upload raw data to S3 bucket in 'data' folder
for i in range(3):
    file_path = f'data/goemotions_{i + 1}.csv'

    # Upload to S3
    s3.upload_file(file_path, bucket_name, file_path)

In [9]:
# Load csvs in order to save in new folder as tsv (to avoid comma issues in pyathena)
first_data_obj = s3.get_object(Bucket=bucket_name, Key='data/goemotions_1.csv')
second_data_obj = s3.get_object(Bucket=bucket_name, Key='data/goemotions_2.csv')
third_data_obj = s3.get_object(Bucket=bucket_name, Key='data/goemotions_3.csv')
df_1 = pd.read_csv(io.BytesIO(first_data_obj['Body'].read()))
df_2 = pd.read_csv(io.BytesIO(second_data_obj['Body'].read()))
df_3 = pd.read_csv(io.BytesIO(third_data_obj['Body'].read()))

In [10]:
dfs = [df_1, df_2, df_3]

for i, df in enumerate(dfs):
    local_file_path = f'/tmp/goemotions_{i + 1}.csv'
    s3_file_path = f'full-dataset/preprocessed_goemotions_{i + 1}.tsv'

    # Save to CSV without extra quoting
    df.to_csv(local_file_path, sep='\t', index=False)

    # Upload to S3
    s3.upload_file(local_file_path, bucket_name, s3_file_path)

In [11]:
region = 'us-west-2'
conn = connect(s3_staging_dir=f"s3://{bucket_name}/athena/results", region_name=region)

In [12]:
database_name = 'emotions_database'
cursor = conn.cursor()
statement = f"CREATE DATABASE IF NOT EXISTS {database_name}"
cursor.execute(statement)

In [13]:
# Filter dataset using pyathena, so that we keep only those samples in which more than one annotator agreed on a label
table_name = 'emotions_table'
create_table_query = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name} (
    text string,
    id string,
    author string,
    subreddit string,
    link_id string,
    parent_id string,
    created_utc bigint,
    rater_id int,
    example_very_unclear boolean,
    admiration int,
    amusement int,
    anger int,
    annoyance int,
    approval int,
    caring int,
    confusion int,
    curiosity int,
    desire int,
    disappointment int,
    disapproval int,
    disgust int,
    embarrassment int,
    excitement int,
    fear int,
    gratitude int,
    grief int,
    joy int,
    love int,
    nervousness int,
    optimism int,
    pride int,
    realization int,
    relief int,
    remorse int,
    sadness int,
    surprise int,
    neutral int
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t'
LINES TERMINATED BY '\n'
LOCATION 's3://{bucket_name}/full-dataset'
TBLPROPERTIES ('skip.header.line.count'='1');
"""

try:
    # Print the database and table names
    print(f"Database: {database_name}, Table: {table_name}")

    # Execute the create table query
    cursor.execute(create_table_query)
    print("Table created successfully.")
except Exception as e:
    print("Error creating table:", e)

Database: emotions_database, Table: emotions_table
Table created successfully.


In [14]:
# Query to group by text, and sum up the different annotators ratings
# Then only keep the sample if more than 1 annotator selected an emotion
COMPARE_RATERS = f"""
SELECT id,
SUM(admiration),
SUM(amusement),
SUM(anger),
SUM(annoyance),
SUM(approval),
SUM(caring),
SUM(confusion),
SUM(curiosity),
SUM(desire),
SUM(disappointment),
SUM(disapproval),
SUM(disgust),
SUM(embarrassment),
SUM(excitement),
SUM(fear),
SUM(gratitude),
SUM(grief),
SUM(joy),
SUM(love),
SUM(nervousness),
SUM(optimism),
SUM(pride),
SUM(realization),
SUM(relief),
SUM(remorse),
SUM(sadness),
SUM(surprise),
SUM(neutral)
FROM {database_name}.{table_name}
GROUP BY id
HAVING SUM(admiration) > 1 OR
       SUM(amusement) > 1 OR
       SUM(anger) > 1 OR
       SUM(annoyance) > 1 OR
       SUM(approval) > 1 OR
       SUM(caring) > 1 OR
       SUM(confusion) > 1 OR
       SUM(curiosity) > 1 OR
       SUM(desire) > 1 OR
       SUM(disappointment) > 1 OR
       SUM(disapproval) > 1 OR
       SUM(disgust) > 1 OR
       SUM(embarrassment) > 1 OR
       SUM(excitement) > 1 OR
       SUM(fear) > 1 OR
       SUM(gratitude) > 1 OR
       SUM(grief) > 1 OR
       SUM(joy) > 1 OR
       SUM(love) > 1 OR
       SUM(nervousness) > 1 OR
       SUM(optimism) > 1 OR
       SUM(pride) > 1 OR
       SUM(realization) > 1 OR
       SUM(relief) > 1 OR
       SUM(remorse) > 1 OR
       SUM(sadness) > 1 OR
       SUM(surprise) > 1 OR
       SUM(neutral) > 1
"""

test = f"""
SELECT *
FROM {database_name}.{table_name}
LIMIT 10;
"""

test2 = f"""
SELECT admiration 
FROM {database_name}.{table_name}
WHERE TRY_CAST(admiration AS INTEGER) IS NULL;
"""
df_filtered = pd.read_sql(COMPARE_RATERS, conn)
df_filtered.head()

/tmp/ipykernel_20/2848378796.py:76: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_filtered = pd.read_sql(COMPARE_RATERS, conn)


,id,_col1,_col2,_col3,_col4,_col5,_col6,_col7,_col8,_col9,...,_col19,_col20,_col21,_col22,_col23,_col24,_col25,_col26,_col27,_col28
0,eczuekb,2,0,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,ef961hv,2,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ed9w1hm,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,2
3,ee52cjs,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,edsqvyx,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [15]:
len(df_filtered)

54263

In [16]:
df_full = pd.concat([df_1, df_2, df_3])

In [17]:
# Create a dictionary from df_full for fast lookup
id_text_dict = pd.Series(df_full['text'].values, index=df_full['id']).to_dict()

# Create a dataframe with only the rows contained in df_filtered, but including the text column from the original data
rows = []

for index, row in df_filtered.iterrows():
    emotions = [int(col.replace('_col', '')) - 1 for col in df_filtered.columns if col != 'id' and row[col] > 1]
    # Use the dictionary for fast lookup
    text = id_text_dict.get(row['id'], '')
    rows.append({'text': text, 'emotions': emotions, 'id': row['id']})

final_df = pd.DataFrame(rows)
final_df.head()        

,text,emotions,id
0,"He isn't as big, but he's still quite popular....",[0],eczuekb
1,that's adorable asf,[0],ef961hv
2,"I have, and now that you mention it, I think t...",[27],ed9w1hm
3,"I wanted to downvote this, but it's not your f...",[27],ee52cjs
4,Build a wall? /jk,[27],edsqvyx


In [18]:
# Reduce multiple labels to a single label for modeling purposes
final_df['emotions'] = final_df['emotions'].apply(lambda x: x[0] if x else None)

In [19]:
local_path = 'data/pre_processed_data.tsv'
final_df.to_csv(local_path, sep='\t', index=False)

# Upload to S3
s3.upload_file(local_path, bucket_name, local_path)